# Recitation 12

It's our final recitation together! It has been a pleasure to be your TA this semester, and though there were some ups and downs, I hope you were able to learn something from the class! Please make sure to fill out the SIRS Survey here: https://rutgers.bluera.com/rutgers. For our final week we'll end with a fun topic, JSON (Java Script Object Notation) and how it works with APIs in Python

From Halloween 2023

<img src="https://rutgers.box.com/shared/static/nsq4kletv5mli8ibpxm0dn3oee379t6k.jpg" alt="halloween-picture" width="250"/>

## Gotta Catch em All!

You've been recently hired as Professor Oak's Lead Data Science Pokemon Researcher congrats! He needs to push a software update to the pokedexes in the Canto region, and he needs your help to get updated information. Use the documentation from the PokeAPI here -> https://pokeapi.co/docs/v2 to help you do the following tasks. *Hint*: Make sure to pretty print when you are printing a json object

- Retrieve information for the following pokemon, then store your info in a dict and write it to a JSON file. Starter code is given below
    1. **Charizard** (Fire/Flying) - Kanto Region
    2. **Lanturn** (Water/Electric) - Johto Region
    3. **Sceptile** (Grass) - Hoenn Region
    4. **Lucario** (Fighting/Steel) - Sinnoh Region
    5. **Chandelure** (Ghost/Fire) - Unova Region
    6. **Greninja** (Water/Dark) - Kalos Region
    7. **Decidueye** (Grass/Ghost) - Alola Region
    8. **Corviknight** (Flying/Steel) - Galar Region
    9. **Boltund** (Electric) - Galar Region
    10. **Silicobra** (Ground) - Galar Region
- Read from the JSON file that you have just saved, then find the strongest move that each pokemon has the ability to learn, pay attention to the `url` key in the moves list
- Your favorite pokemon trainer Ash Ketchum is currently traveling through multiple regions looking for the above pokemon, use the `location_area_encounters` key in each pokemon and check if any of the pokemon can be found and where they can be found. Save this file to be sent to Ash.
- Lastly, you want to know how to get every pokemon in the list. Loop through the pokemon and check if said pokemon has a previous evolution, then use the `trigger` key to figure out how to evolve the pokemon. Store this information in a new dictionary. Then print the evolution methods in a nice print statement

<img src="https://media.tenor.com/4s6S60RxJacAAAAd/wooper-pokemon.gif" alt="halloween-picture" width="250"/>

In [ ]:
# Query the api for each pokemon, save json in a dict, write the file to your folder

### ENTER CODE BELOW

import requests
import json

pokemon = ["charizard", "lanturn", "sceptile", "lucario", "chandelure", "greninja", "decidueye", "corviknight", "boltund", "silicobra"]

url = "https://pokeapi.co/api/v2/pokemon/"
response = requests.get().json()
poke_data = json.loads(response)

# print(json.dumps(json_object, indent = 1)) <- pretty printing

### Answer

```python

import requests
import json

pokemon = ["charizard", "lanturn", "sceptile", "lucario", "chandelure", "greninja", "decidueye", "corviknight", "boltund", "silicobra"]
poke_data = {}

for p in pokemon:
    url = "https://pokeapi.co/api/v2/pokemon/" + p
    try:
        response = requests.get(url).json()
        poke_data.update({p: json.loads(response)})
    except:
        print("API Error " + url)

with open('./poke_data.json', 'w') as json_file:
    json.dump(poke_data, json_file)
```

In [ ]:
# Read from the JSON file that you have just saved, then find the strongest move that each pokemon has the ability to learn, 
# pay attention to the `url` key in the moves list

### ENTER CODE BELOW

### Answer

```python

import json
import requests

def get_strongest_move(pokemon_data):
    moves = pokemon_data.get('moves', [])
    
    if not moves:
        print("Could not get moves")
        return None
    
    # Initialize variables to store the strongest move information
    strongest_move_name = None
    strongest_move_power = 0
    
    for m in moves:
        move = m['move']
        move_url = move['url']
        if move_url:
            move_data = requests.get(move_url).json()
            move_power = move_data.get('power', 0)

            # handle None value
            if move_power:
                move_power = int(move_power)
            else:
                move_power = 0
            
            # Update the strongest move if the current move is more powerful
            if move_power > strongest_move_power:
                strongest_move_power = move_power
                strongest_move_name = move_data.get('name', 'Unknown')
    
    return strongest_move_name, strongest_move_power

# Load the saved Pokemon data from the JSON file
file_path = './poke_data.json'
with open(file_path, 'r') as json_file:
    poke_data = json.load(json_file)

# Iterate through each Pokemon and find the strongest move
for poke_name, data in poke_data.items():
    move_name, power = get_strongest_move(data)
    print(f"{poke_name}'s strongest move is {move_name} with power {power}")
```

**Q**: Why does your code take so long? Do you notice how a lot of the pokemon share the moves that they can learn? How can we make this code faster by saving certain moves locally?

**Q**: How can we change the code to find the second best move? Third best?


In [ ]:
# Your favorite pokemon trainer Ash Ketchum is currently traveling through multiple regions looking for the above pokemon, use the 
# `location_area_encounters` key in each pokemon and check if any of the pokemon can be found and where they can be found. Save this file to be sent to Ash.

### ENTER CODE BELOW

```python

locations_info = {}

for poke_name, data in poke_data.items():
    location_info = requests.get(data['location_area_encounters']).json()
    versions = set()
    locations = []
    if location_info:
        for l in location_info:
            for v in l['version_details']:
                versions.add(v['version']['name'])
            locations.append(l['location_area']['name'])
    locations_info.update({poke_name: {'game_versions': list(versions), 'locations': locations}})
                

with open("./poke_locations.json", 'w') as json_file:
    json.dump(locations_info, json_file)
```

In [ ]:
# Lastly, you want to know how to get every pokemon in the list. Loop through the pokemon and check if said pokemon has a 
# previous evolution, then use the `trigger` key to figure out how to evolve the pokemon. Store this information in a new 
# dictionary. Then print the evolution methods in a nice print statement

### ENTER CODE BELOW

### Answer

```python

import requests
import json


def get_evolution_chain(pokemon_species_url):
    response = requests.get(pokemon_species_url)
    data = response.json()

    evolution_chain_url = data['evolution_chain']['url']
    evolution_chain_data = requests.get(evolution_chain_url).json()

    return evolution_chain_data['chain']


def extract_evolution_info(evolution_chain, evolution_info=None):
    if evolution_info is None:
        evolution_info = {}

    species_name = evolution_chain['species']['name']
    evolution_info[species_name] = {}

    for evolution_detail in evolution_chain.get('evolution_details', []):
        trigger_name = evolution_detail['trigger']['name']
        min_level = evolution_detail.get('min_level', None)
        item = evolution_detail.get('item', None)

        evolution_info[species_name][trigger_name] = {
            'min_level': min_level,
            'item': item
        }

    for evolves_to in evolution_chain.get('evolves_to', []):
        extract_evolution_info(evolves_to, evolution_info[species_name])

    return evolution_info

evolution_info = {}
for poke_name, data in poke_data.items():
    evolution_chain_data = get_evolution_chain(data['species']['url'])
    extract_evolution_info(evolution_chain_data, evolution_info)

print(json.dumps(evolution_info, indent=1))
```

**Q**: Why do we use a recursive function to retrieve all of the data? How can we avoid using recursion in this situation? *Hint* look up how to flatten JSON in python

### Flattned Version

```python

import requests
import json


def get_evolution_chain(pokemon_species_url):
    response = requests.get(pokemon_species_url)
    data = response.json()

    evolution_chain_url = data['evolution_chain']['url']
    evolution_chain_data = requests.get(evolution_chain_url).json()

    return evolution_chain_data['chain']


def extract_evolution_info(evolution_chain, evolution_info=None):
    if evolution_info is None:
        evolution_info = []

    species_name = evolution_chain['species']['name']
    curr_evolution_info = {}
    curr_evolution_info[species_name] = {}

    for evolution_detail in evolution_chain.get('evolution_details', []):
        trigger_name = evolution_detail['trigger']['name']
        min_level = evolution_detail.get('min_level', None)
        item = evolution_detail.get('item', None)

        curr_evolution_info[species_name][trigger_name] = {
            'min_level': min_level,
            'item': item
        }
    evolution_info.append(curr_evolution_info)
    for evolves_to in evolution_chain.get('evolves_to', []):
        extract_evolution_info(evolves_to, evolution_info)

    return evolution_info

comp_evolution_info = []
for poke_name, data in poke_data.items():
    evolution_info = []
    evolution_chain_data = get_evolution_chain(data['species']['url'])
    comp_evolution_info.append(extract_evolution_info(evolution_chain_data, evolution_info))

evolution_lines = []
for c in comp_evolution_info:
    e_l = []
    for p in c:
        for k, v in p.items():
            e_l.append(k)
    evolution_lines.append(e_l)

```

## Your Restaurant Critic Era

You might be surprised to hear it, but JSON files work perfectly fine with pandas! For this task we will work with the yelp_businesses.json file.
- The file has a small issue however, it needs to be read in line by line, go ahead and do that first, populate each line into a list called `data`
- Read the data using pandas and name the dataframe `yelp_df`
- If you call the `head` function on the dataframe, you'll see that there is nested data in the attributes column, you can work with nested data in pandas using the `json_normalize()` function from pandas

In [ ]:
!curl -L  https://rutgers.box.com/shared/static/wtt1oz9nw8h0xeuztekj45dghhud37uz --output yelp_businesses.json
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
# The file has a small issue however, it needs to be read in line by line, go ahead and do that first, populate each line 
# into a list called `data`

### ENTER CODE BELOW

### Answer
```python

with open("./yelp_businesses.json", 'r') as data_file:
    data = []
    for line in data_file:
        data.append(json.loads(line))
```

In [ ]:
# Read the data using pandas and name the dataframe `yelp_df`

### ENTER CODE BELOW

### Answer
```python

yelp_df = pd.DataFrame(data)
```

In [ ]:
# Normalize the json data

### ENTER CODE BELOW

### Answer
```python
yelp_df = pd.json_normalize(data)
yelp_df.head()
```

Since this is the last recitation, I'll take it easy and call it a day. This is a good time to recap the semester, please ask any questions that you might have lingering. Additionally, it's a good time to go through previous recitation notebooks and check your understanding on all of them (except recitation_11 since that was more of a tangential one). Have a great winter break y'all!